In [1]:
from shearnet.core.dataset import generate_dataset, split_combined_images

from shearnet.utils.metrics import calculate_ngmix_response_matrix, calculate_multiplicative_bias_ngmix

import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from typing import Optional

In [2]:
"""
Generate the dataset
"""

test_target_images, test_target_labels, obs = generate_dataset(
        samples=5000,
        psf_sigma=0.25,
        type='gauss',
        exp='ideal',
        seed=42,
        nse_sd=1e-5,
        npix=53,
        scale=0.141,
        return_psf=True,
        return_clean=False,
        return_obs=True,
    )

test_galaxy_images, test_psf_images = split_combined_images(test_target_images, has_psf=True, has_clean=False)

100%|██████████| 5000/5000 [00:15<00:00, 331.76it/s]


In [3]:
"""
Calculate response matrix
"""

R, R_per_galaxy = calculate_ngmix_response_matrix(obs, test_target_labels, h=0.01, seed=1234, psf_model='gauss', gal_model='gauss')

print(R)


Calculating NGmix Response Matrix...
Running NGmix on e1_positive images...
Starting NGmix ML fitting: num_gal: 5000 | psf_model: gauss | gal_model: gauss | num_cores: 96
Running NGmix on e1_negative images...
Starting NGmix ML fitting: num_gal: 5000 | psf_model: gauss | gal_model: gauss | num_cores: 96
Running NGmix on e2_positive images...
Starting NGmix ML fitting: num_gal: 5000 | psf_model: gauss | gal_model: gauss | num_cores: 96
Running NGmix on e2_negative images...
Starting NGmix ML fitting: num_gal: 5000 | psf_model: gauss | gal_model: gauss | num_cores: 96
Valid galaxies after removing NaNs: 5000/5000

NGmix Response Matrix (averaged over 5000 galaxies):
R = [[1.018302, -0.004317],
     [0.007021, 0.987071]]
[[ 1.01830215 -0.00431715]
 [ 0.0070212   0.98707099]]


In [4]:
"""
Generate multiplicative bias datasets
"""

test_target_images_g1_pos, __, obs_g1_pos = generate_dataset(
        samples=5000,
        psf_sigma=0.25,
        type='gauss',
        exp='ideal',
        seed=42,
        nse_sd=1e-5,
        npix=53,
        scale=0.141,
        return_psf=False,
        return_clean=True,
        return_obs=True,
        base_shear_g1=0.02,
        base_shear_g2=0.0
    )
test_target_images_g1_neg, __, obs_g1_neg = generate_dataset(
        samples=5000,
        psf_sigma=0.25,
        type='gauss',
        exp='ideal',
        seed=42,
        nse_sd=1e-5,
        npix=53,
        scale=0.141,
        return_psf=False,
        return_clean=True,
        return_obs=True,
        base_shear_g1=-0.02,
        base_shear_g2=0.0
    )
test_target_images_g2_pos, __, obs_g2_pos = generate_dataset(
        samples=5000,
        psf_sigma=0.25,
        type='gauss',
        exp='ideal',
        seed=42,
        nse_sd=1e-5,
        npix=53,
        scale=0.141,
        return_psf=False,
        return_clean=True,
        return_obs=True,
        base_shear_g1=0.0,
        base_shear_g2=0.02
    )
test_target_images_g2_neg, __, obs_g2_neg = generate_dataset(
        samples=5000,
        psf_sigma=0.25,
        type='gauss',
        exp='ideal',
        seed=42,
        nse_sd=1e-5,
        npix=53,
        scale=0.141,
        return_psf=False,
        return_clean=True,
        return_obs=True,
        base_shear_g1=0.0,
        base_shear_g2=-0.02
    )

test_galaxy_images_g1_pos, test_psf_images_g1_pos = split_combined_images(test_target_images_g1_pos, has_psf=True, has_clean=False)
test_galaxy_images_g1_neg, test_psf_images_g1_neg = split_combined_images(test_target_images_g1_neg, has_psf=True, has_clean=False)
test_galaxy_images_g2_pos, test_psf_images_g2_pos = split_combined_images(test_target_images_g2_pos, has_psf=True, has_clean=False)
test_galaxy_images_g2_neg, test_psf_images_g2_neg = split_combined_images(test_target_images_g2_neg, has_psf=True, has_clean=False)

100%|██████████| 5000/5000 [00:14<00:00, 350.82it/s]


In [5]:
"""
Calculate multiplicative bias
"""

statistics = calculate_multiplicative_bias_ngmix(obs_g1_pos=obs_g1_pos, obs_g1_neg=obs_g1_neg, obs_g2_pos=obs_g2_pos, obs_g2_neg=obs_g2_neg, true_shear_step=0.02, h=0.01, seed=1234, psf_model='gauss', gal_model='gauss')

print('m1: '+str(statistics['m1']))
print('c1: '+str(statistics['c1']))
print('m2: '+str(statistics['m2']))
print('c2: '+str(statistics['c2']))


CALCULATING MULTIPLICATIVE AND ADDITIVE BIAS (NGMIX)
True shear: ±0.02
Response perturbation: ±0.01

--- Component 1 (g1) ---

Dataset A (g1 = +0.02):

Calculating NGmix Response Matrix...
Running NGmix on e1_positive images...
Starting NGmix ML fitting: num_gal: 5000 | psf_model: gauss | gal_model: gauss | num_cores: 96
Running NGmix on e1_negative images...
Starting NGmix ML fitting: num_gal: 5000 | psf_model: gauss | gal_model: gauss | num_cores: 96
Running NGmix on e2_positive images...
Starting NGmix ML fitting: num_gal: 5000 | psf_model: gauss | gal_model: gauss | num_cores: 96
Running NGmix on e2_negative images...
Starting NGmix ML fitting: num_gal: 5000 | psf_model: gauss | gal_model: gauss | num_cores: 96
Valid galaxies after removing NaNs: 5000/5000

NGmix Response Matrix (averaged over 5000 galaxies):
R = [[1.032341, 0.000790],
     [-0.013603, 0.970191]]

Dataset B (g1 = -0.02):

Calculating NGmix Response Matrix...
Running NGmix on e1_positive images...
Starting NGmix ML

In [6]:
"""
Trying ShearNet as well
"""

from shearnet.utils.metrics import calculate_response_matrix, calculate_multiplicative_bias

from shearnet.cli.evaluate import load_config, create_parser, initialize_model

parser = create_parser()
args = parser.parse_args(["--model_name", "fork-like_ideal_low-noise", "--test_samples", "5000"])
config = load_config(args)

state = initialize_model(config, test_galaxy_images, test_psf_images)

R, R_per_galaxy = calculate_response_matrix(state=state, observations=obs, batch_size=32, h=0.01, model_type='fork', psf_images=test_psf_images)

statistics_nn = calculate_multiplicative_bias(state=state, obs_g1_pos=obs_g1_pos, obs_g1_neg=obs_g1_neg, obs_g2_pos=obs_g2_pos, obs_g2_neg=obs_g2_neg, true_shear_step=0.02, batch_size=32, h=0.01, model_type='fork', psf_g1_pos=test_psf_images_g1_pos, psf_g1_neg=test_psf_images_g1_neg, psf_g2_pos=test_psf_images_g2_pos, psf_g2_neg=test_psf_images_g2_neg)

print('m1: '+str(statistics_nn['m1']))
print('c1: '+str(statistics_nn['c1']))
print('m2: '+str(statistics_nn['m2']))
print('c2: '+str(statistics_nn['c2']))


Loading model config from: /home/adfield/ShearNet/plots/fork-like_ideal_low-noise/training_config.yaml

Evaluation Configuration

evaluation:
  test_samples: 5000
  seed: 58

model:
  process_psf: True
  type: fork-like
  galaxy: {'type': 'research_backed'}
  psf: {'type': 'forklens_psf'}

plotting:
  plot: True

comparison:
  mcal: True
  ngmix: True
  psf_model: gauss
  gal_model: gauss


Loading Model
Found 1 matching checkpoint(s):
  1. fork-like_ideal_low-noise300

Loading checkpoint from: /home/adfield/ShearNet/model_checkpoint/fork-like_ideal_low-noise300
✓ Model checkpoint loaded successfully

Calculating Response Matrix...
Extracted 5000 galaxies with sheared images
Shear step: ±0.01

Response Matrix (averaged over 5000 galaxies):
R = [[1.003655, -0.002884],
     [-0.002671, 0.995212]]

Response matrix statistics:
  R_11: mean=1.003655, std=0.142875
  R_22: mean=0.995212, std=0.142421
  R_12: mean=-0.002884, std=0.138087
  R_21: mean=-0.002671, std=0.138451

CALCULATING MULTI